#### 🧠 理解问题  

> *让客流量随机分配在一年中的12个月里，但需要保证某几个月的客流量为0，而另外的几个月里客流量要显著高于其他的月份。*  

该问题实际上就是一个概率分布问题，只需要让客流量为0的月份里概率为0，而客流量较高的月份里概率更高即可。  
当然，关于**较高**是怎么个高法，我们仍然有办法去控制。  
以下给出上述算法实例。  

#### 🔢 算法描述  

\begin{array}{ll}
n : \text{总体值} & P_{\text{unit}} : \text{基础概率单位} = \frac{1}{W + p \times |H|} \\
Z : \text{客流量为0的月份的集合} &  P_i : \text{第}i\text{个月的概率} = \begin{cases} 
0 & \text{if } i \in Z \\
P_{\text{unit}} \times p & \text{if } i \in H \\
P_{\text{unit}} & \text{otherwise} 
\end{cases} \\
H : \text{客流量较高的月份的集合} & \Delta : \text{随机扰动量} = \text{Uniform}(-\delta, \delta) \\
p : \text{业绩参数} & P'_{i} : \text{施加扰动后的概率} = \begin{cases} 
P_i & \text{if Purging and } P_i = 0 \\
\text{Clip}(P_i + \Delta_i, 0, 1) & \text{otherwise} 
\end{cases} \\
\delta : \text{扰动参数} & P''_{i} : \text{归一化后的概率} = \frac{P'_{i}}{\sum P'_{i}} \\
\text{Purging} : \text{是否保持原为0的概率}&  \\
\end{array}

In [1]:
import numpy as np


def yzw_distribution(
    total: int, 
    zero_months: list[int], 
    higher_months: list[int], 
    performance: float, 
    noise: float = 0.05, 
    purging: bool = False
) -> list[int]:
    """志伟分布（套皮多项分布）：研究如何将总体值按指定规则分配到12个月份中。

    :param total: 总体值（总客流量）。
    :param zero_months: 客流量为0的月份。
    :param higher_months: 客流量较高的月份。
    :param performance: 业绩参数，用于控制客流量较高的月份高于其他月份的程度。如果该值小于1，则会起到相反的效果。
    :param noise: 扰动（噪声），使得数据分布显得更加不规律。当设置了较高的performance时，应将此值调低。
    :param purging: 如果将此项置为True，则客流量为0的月份不受扰动，保持为0。
    :return: 客流量实际分布。
    """
    normal_months_number = 12 - (len(zero_months) + len(higher_months))
    weight = normal_months_number + performance * len(higher_months)
    probs_per_unit = 1.0 / weight

    probs_list = []
    for this_month in range(12):
        if this_month in zero_months:
            probs_list.append(0)
        elif this_month in higher_months:
            probs_list.append(probs_per_unit * performance)
        else:
            probs_list.append(probs_per_unit * 1.0)
    probs_list = np.array(probs_list)

    noise_list = np.random.uniform(-noise, noise, size=len(probs_list))
    if purging:
        noise_list[probs_list == 0] = 0
    probs_list_with_noise = np.array(probs_list) + noise_list # 施加扰动
    probs_list_with_noise = np.clip(probs_list_with_noise, 0, 1) # 裁剪，确保每个概率都在[0,1)区间内
    probs_list_with_noise = probs_list_with_noise / probs_list_with_noise.sum() # 归一化

    return np.random.multinomial(total, probs_list_with_noise)

#### 😀 函数定义完成！让我们开始测试！

|参数|值|描述|
|---|---|---|
|total|500000|假设客流量是50万人|
|zero_months|[7, 9]|假设8月和10月客流量为0；从0开始计数|
|higher_months|[2, 3, 4]|假设3到5月客流量较高|
|performance|1.5|假设客流量较高的月份，概率是其他月份的1.5倍|
|noise|0.01|扰动0.01|
|purging|False|默认值，使得客流量为0的月份同样收到扰动|

⚠ **注意：由于概率分布的固有性质，即使所有参数相同，每次运行都会得到不一样的结果**

In [2]:
from plotting_tool import plot


test_total = 500000
test_zero_months = [7, 9]
test_higher_months = [2, 3, 4]
test_performance = 1.5
test_noise = 0.01
test_purging = False

plot(yzw_distribution, test_total, test_zero_months, test_higher_months, test_performance, test_noise, test_purging)

#### 👣 行百里者半九十  

到这里我们绝大多数工作都已经完成，然而还有一件最重要的事情没有做——`将数据自动写入Excel`。  
我们当然可以通过算法计算出数据，然后手动填进去，但那样做和纯手动又有多大区别的？  

#### ✍ 自动写入Excel

因此，最后一部分便是：

> 如何在**不改变原Excel格式和其他数据**的情况下，将我们需要的数据写入进去。

ℹ *注意：由于这一部分多是无意义的调库，因此不会对代码作过多解释，建议开箱即用。*

In [3]:
from openpyxl.reader.excel import load_workbook
from collections.abc import Iterable


def write_excel(
    obj: list, 
    file_path: str, 
    sheet_name: str, 
    cells: list
):
    assert isinstance(obj, Iterable), 'Data should be iterable, such as list[] or tuple(). '
    assert isinstance(cells, Iterable), 'Cells should be iterable, such as list[] or tuple(). '
    assert len(obj) == len(cells), 'Number of data should equal to number of cells. '

    workbook = load_workbook(file_path)
    worksheet = workbook[sheet_name]
    for i, cell in enumerate(cells):
        col = ord(cell[0].upper()) - 64
        row = int(cell[1:])
        worksheet.cell(row, col).value = obj[i]
    workbook.save(file_path)

#### 🤫 不废话，直接看实例

In [4]:
from example_recovery import example_recover


example_recover() # 还原示例表，以防被之前的操作影响

data_to_write = yzw_distribution(test_total, test_zero_months, test_higher_months, test_performance, test_noise, test_purging)
# 将所有12个月数据写在一张Sheet上
cells = [f'B{row}' for row in range(3, 15)]
# 也就是 cells = ['B3', 'B4', ... , 'B14']
write_excel(data_to_write, 'example.xlsx', 'Sheet1', cells)

--2023-11-02 09:43:54--  https://src.mealuet.com/example.xlsx
Resolving src.mealuet.com (src.mealuet.com)... 104.21.94.150, 172.67.137.58, 2606:4700:3031::ac43:893a, ...
Connecting to src.mealuet.com (src.mealuet.com)|104.21.94.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17396 (17K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘example.xlsx’

     0K .......... ......                                     100% 74.9M=0s

2023-11-02 09:43:54 (74.9 MB/s) - ‘example.xlsx’ saved [17396/17396]



In [5]:
example_recover()

data_to_write = yzw_distribution(test_total, test_zero_months, test_higher_months, test_performance, test_noise, test_purging)
# 将所有12个月的数据写在一张Sheet上，但是单元格不连续
cells = [f'B{row}' for row in range(4, 15)]
cells.append('C39')
# 也就是 cells = ['B4', 'B5', ... , 'B14', 'C39']
write_excel(data_to_write, 'example.xlsx', 'Sheet1', cells)

--2023-11-02 09:43:57--  https://src.mealuet.com/example.xlsx
Resolving src.mealuet.com (src.mealuet.com)... 172.67.137.58, 104.21.94.150, 2606:4700:3031::6815:5e96, ...
Connecting to src.mealuet.com (src.mealuet.com)|172.67.137.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17396 (17K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘example.xlsx’

     0K .......... ......                                     100% 45.8M=0s

2023-11-02 09:43:57 (45.8 MB/s) - ‘example.xlsx’ saved [17396/17396]



In [6]:
example_recover()


data_to_write = yzw_distribution(test_total, test_zero_months, test_higher_months, test_performance, test_noise, test_purging)
# 将一部分数据写在Sheet1上，另一部分写在Sheet2上
cells_1 = [f'B{row}' for row in range(3, 12)]
data_1 = data_to_write[:9]
# data和cell的数量必须相等——这很好理解
cells_2 = [f'C{row}' for row in range(16, 19)]
data_2 = data_to_write[9:]
# 调用两次write_excel，写两次即可
write_excel(data_1, 'example.xlsx', 'Sheet1', cells_1)
write_excel(data_2, 'example.xlsx', 'Shit2', cells_2)
# 以此类推，如果写在多张表上就像这样切分一下，然后运行若干次write_excel即可
# 写在不同的Excel文件里，则修改参数example.xlsx即可

--2023-11-02 09:43:59--  https://src.mealuet.com/example.xlsx
Resolving src.mealuet.com (src.mealuet.com)... 172.67.137.58, 104.21.94.150, 2606:4700:3031::ac43:893a, ...
Connecting to src.mealuet.com (src.mealuet.com)|172.67.137.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17396 (17K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘example.xlsx’

     0K .......... ......                                     100% 81.5M=0s

2023-11-02 09:44:00 (81.5 MB/s) - ‘example.xlsx’ saved [17396/17396]



---

#### 本课题到此就算结束啦，辛苦！
#### 开箱即用的代码保存在`main.py`里，下载下来直接使用即可
#### 使用前记得先安装必要的软件包

```bash
pip3 install -r requirements.txt
```